# Hn Stage to OMOP

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
hnStage_IDEA4RC = pd.read_csv("./IDEA4RC-data/hnStageIDEA4RC.csv")
hnStage_IDEA4RC.head(5)

In [ ]:
# Conexión a la base de datos OMOP CDM
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

# Comando para hacer el mapeo (insercción de datos del IDEA4RC.csv a OMOP)
cur = conn.cursor()
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

### Hn Stage to Measurement

For the following:

Soft tissue
distant lymph node
lung
bone
liver
pleura
peritoneum
brain
other viscera
unknown

I really do not know what to do, since it is expected to be a boolean, but then what do I do with the Vocabulary????

In [2]:
df_tables=hnStage_IDEA4RC

cols = {
    "cT" : 0,
    "cN" : 0,
    "cM" : 0,
    "pT" : 0,
    "pN" : 0,
    "pM" : 0,
    "Radiological Extra-nodal extension (rEne)" : 0,
    "Staging classification" : 0,
    "Soft tissue" : 32945,
    "distant lymph node" : 32946,
    "lung" : 32947,
    "bone" : 32948,
    "liver" : 32949,
    "pleura" : 32950,
    "peritoneum" : 32951,
    "brain" : 32952,
    "other viscera" : 32952,
    "unknown" : 32952
}
sql = """
    INSERT INTO omopcdm.measurement (person_id, measurement_concept_id, measurement_date,measurement_type_concept_id,value_as_concept_id,measurement_event_id)
    VALUES (%s, %s, %s,%s,%s,%s)
"""

query= """
    SELECT c.patient 
    FROM cancer_episode c
    WHERE EXISTS (
        SELECT 1 
        FROM EpisodeEvent e
        WHERE e.cancerEpisode = c.id
        AND e.id = %s
    )
    LIMIT 1

"""
queryDate="""
    SELECT episode_start_date
    FROM cancer_episode
    WHERE id = %s
"""
sqlGetEpisode = """
    SELECT c.id 
    FROM episode c
    WHERE c.person_id = %s
    AND c.episode_start_date = %s
    LIMIT 1
"""

booleans=[
    "Soft tissue",
    "distant lymph node",
    "lung",
    "bone",
    "liver",
    "pleura",
    "peritoneum",
    "brain",
    "other viscera",
    "unknown"
]
boolValues=[4188539,4188540]

person_id=0 
date=datetime.now()
measurement_type_concept_id=38000280

for idx, row in df_tables.iterrows():
    for column in cols.keys():
        curIDEA.execute(query, (row['Episode Event Reference']))
        resIDEA=cur.fetchone()
        person_id=res
        measurement_value = row[column]
        if((column in booleans) and (measurement_value not in boolValues)):
            if measurement_value==0:
                measurement_value=4188540
            else:
                measurement_value=4188539
        measurement_concept_id = cols.get(column)
        curIDEA.execute(queryDate, (row['Episode Event Reference']))
        res=curIDEA.fetchone()
        dateAux=res
        cur.execute(sqlGetEpisode, (person_id,dateAux))
        measurement_event_id=cur.fetchone()
        cur.execute(sql, (person_id, measurement_concept_id, date, measurement_type_concept_id,measurement_value,measurement_event_id))
        conn.commit()



NameError: name 'hnStage_IDEA4RC' is not defined

In [ ]:
1147138 este es el id de la columna episode_event_field_concept_id en episode_event
que nos va a redirigir a la columna adecuada en measurement